In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

https://drive.google.com/file/d/1dLAE0W2mkAoYESoOC2Z3P4f4ro8TPEkZ/view?usp=drive_link

# Load data


In [ ]:
data01_df = pd.read_csv('transfermarkt_fbref_201718.csv')
data02_df = pd.read_csv('transfermarkt_fbref_201819.csv')
data_df = pd.read_csv('transfermarkt_fbref_201920.csv')
data_df = pd.concat([data_df, data01_df])
data_df = pd.concat([data_df, data02_df])
condition = (data_df['age'] > 1000) | (data_df['age'] == 0)
data_df = data_df.drop(data_df[condition].index, axis=0)
data_df['height'] = pd.to_numeric(data_df['height'], errors='coerce')
condition = (data_df['height'] == 0)
data_df = data_df.drop(data_df[condition].index, axis=0)

data_df['CLBestScorer'] = data_df['CLBestScorer'].fillna(0)
data_df = data_df.fillna(0)
data_df = data_df.drop('xG.1', axis=1)
# data_df.head()

<ipython-input-4-1024517dd679>:2: DtypeWarning: Columns (395) have mixed types. Specify dtype option on import or set low_memory=False.
  data02_df = pd.read_csv('transfermarkt_fbref_201819.csv')


In [ ]:
temp = data_df.isnull().sum()
temp[temp != 0]


Series([], dtype: int64)

In [ ]:
data_df['value'].describe()

count    6.565000e+03
mean     1.020603e+07
std      1.640225e+07
min      5.000000e+01
25%      1.500000e+06
50%      4.000000e+06
75%      1.200000e+07
max      2.000000e+08
Name: value, dtype: float64

In [ ]:
# mean = data_df['value'].mean()
# std = data_df['value'].std()
# test_df = data_df.copy()
# test_df = test_df[test_df['value'] <= mean + 2*std]
# # test_df = test_df[test_df['value'] >= mean - 2*std]
# # test_df.head()
# data_df = test_df.copy()

In [ ]:
# fig, ax = plt.subplots(figsize=(18,19))
# plt.scatter(data_df['age'], np.log(data_df['value']))
# plt.xlabel('age')
# plt.ylabel('Value')
# plt.title('Scatter plot of Value vs age')
# plt.ticklabel_format(style='plain')
# plt.show()

In [ ]:
data201920_df = data_df.copy()

dummies = pd.get_dummies(data201920_df[['league']], prefix_sep='_')
data201920_df = pd.concat([data201920_df, dummies], axis=1)
data201920_df = data201920_df.rename({"league_Bundesliga":"isBundesliga",
                                "league_La Liga":"isLaLiga",
                                "league_Premier League":"isPremierLeague",
                                "league_Ligue 1":"isLigue1",
                                "league_Serie A":"isSerieA"},axis='columns')
dummies_foot = pd.get_dummies(data201920_df[['foot']], prefix_sep='_')
data201920_df = pd.concat([data201920_df, dummies_foot], axis=1)

data201920_df = data201920_df[data201920_df['value']>1000000]
data201920_df = data201920_df[data201920_df['games']>5]
data201920_df = data201920_df[data201920_df['age']>0]

data201920_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5486 entries, 0 to 2099
Columns: 409 entries, Column1 to foot_right
dtypes: float64(263), int64(126), object(11), uint8(9)
memory usage: 16.8+ MB


In [ ]:
data201920_df = data201920_df.drop(['Season\t\t'], axis=1)
# data201920_df.describe()

In [ ]:
def stepwise_selection(X, y,
               initial_list = [],
               threshold_in = 0.01,
               threshold_out = 0.05,
               verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        # 後向刪除
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(dtype=float,index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        # 找到p-value最小的欄位
        min_pval = new_pval.min()
        if min_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'加入特徵：{best_feature:20}，P-value：{min_pval:.10f}')
        # 前向選擇
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        # 找到p-value最大的欄位
        max_pval = pvalues.max()
        if max_pval > threshold_out:
            changed = True
            worst_feature = pvalues.argmax()
            included.pop(worst_feature)
            if verbose:
                print(f'刪除特徵：{worst_feature:20}，P-value：{max_pval:.10f}')
        if not changed:
            break
    return included

# FWD

In [ ]:
dataFWD = data201920_df[data201920_df['position2'].str[:7]=='Forward']
# dataFWD.head()

In [ ]:
COLUMNS_FWD = ['games', 'Pts', 'goals','passes','dribbles','pressures', 'touches_att_pen_area','tackles','age','ball_recoveries','miscontrols','offsides']

X = dataFWD[COLUMNS_FWD]
y = np.log(dataFWD['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif)
print(results.summary())

    VIF Factor              features
0    75.060609                 const
1     4.327528                 games
2     1.431950                   Pts
3     3.527864                 goals
4     5.602956                passes
5     3.186390              dribbles
6     6.274234             pressures
7     7.068749  touches_att_pen_area
8     3.437220               tackles
9     1.185171                   age
10    9.572302       ball_recoveries
11    5.302794           miscontrols
12    2.829643              offsides
                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.626
Model:                            OLS   Adj. R-squared:                  0.622
Method:                 Least Squares   F-statistic:                     132.1
Date:                Sat, 27 May 2023   Prob (F-statistic):          1.27e-192
Time:                        15:28:55   Log-Likelihood:                -968.15
No. Obse

In [ ]:
COLUMNS_FWD = ['Pts','passes','pressures','tackles','miscontrols','offsides']
COLUMNS_FWD += ['goals', 'dribbles', 'ball_recoveries', 'touches_att_pen_area','gca']

X = dataFWD[COLUMNS_FWD]
y = np.log(dataFWD['value'])
result = stepwise_selection(X, y, COLUMNS_FWD)
print(result)

刪除特徵：                   5，P-value：0.5447515997
刪除特徵：                   9，P-value：0.4387324174
刪除特徵：                   4，P-value：0.1954544539
刪除特徵：                   6，P-value：0.0929481582
['Pts', 'passes', 'pressures', 'tackles', 'goals', 'dribbles', 'touches_att_pen_area']


In [ ]:
result += ['gca','games_starts','age','CL','errors']

X = dataFWD[result]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif)
print(results.summary())

    VIF Factor              features
0    72.338661                 const
1     2.499111                   Pts
2     6.002763                passes
3     5.532567             pressures
4     2.989823               tackles
5     3.868843                 goals
6     2.855959              dribbles
7     6.002133  touches_att_pen_area
8     4.204420                   gca
9     9.483866          games_starts
10    1.172317                   age
11    2.309717                    CL
12    1.047157                errors
                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.628
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     133.1
Date:                Sat, 27 May 2023   Prob (F-statistic):          1.49e-193
Time:                        15:28:55   Log-Likelihood:                -965.97
No. Obse

# 固定效果

In [ ]:
COLUMNS_FWD = ['Pts','passes','tackles','age','goals','dribbles','touches_att_pen_area','gca']
COLUMNS_FWD += ['games_starts','age','CL','errors']
COLUMNS_FWD += ['isSerieA']

X = dataFWD[COLUMNS_FWD]
y = np.log(dataFWD['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.628
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                     133.4
Date:                Sat, 27 May 2023   Prob (F-statistic):          8.03e-194
Time:                        15:28:55   Log-Likelihood:                -965.33
No. Observations:                 959   AIC:                             1957.
Df Residuals:                     946   BIC:                             2020.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.0924 

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [ ]:
COLUMNS_FWD = ['Pts','passes','tackles','age','goals','dribbles','touches_att_pen_area','gca']
COLUMNS_FWD += ['games_starts','age','CL','errors']
COLUMNS_FWD += ['isPremierLeague']

X = dataFWD[COLUMNS_FWD]
y = np.log(dataFWD['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.672
Method:                 Least Squares   F-statistic:                     164.8
Date:                Sat, 27 May 2023   Prob (F-statistic):          5.14e-222
Time:                        15:28:55   Log-Likelihood:                -899.15
No. Observations:                 959   AIC:                             1824.
Df Residuals:                     946   BIC:                             1888.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.0695 

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [ ]:
COLUMNS_FWD = ['Pts','passes','tackles','age','goals','dribbles','touches_att_pen_area','gca']
COLUMNS_FWD += ['games_starts','age','CL','errors']
COLUMNS_FWD += ['isLigue1']

X = dataFWD[COLUMNS_FWD]
y = np.log(dataFWD['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.641
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     141.0
Date:                Sat, 27 May 2023   Prob (F-statistic):          5.11e-201
Time:                        15:28:55   Log-Likelihood:                -948.44
No. Observations:                 959   AIC:                             1923.
Df Residuals:                     946   BIC:                             1986.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.2633 

In [ ]:
COLUMNS_FWD = ['Pts','passes','tackles','age','goals','dribbles','touches_att_pen_area','gca']
COLUMNS_FWD += ['games_starts','age','CL','errors']
COLUMNS_FWD += ['isLaLiga']

X = dataFWD[COLUMNS_FWD]
y = np.log(dataFWD['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.633
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     135.7
Date:                Sat, 27 May 2023   Prob (F-statistic):          4.53e-196
Time:                        15:28:55   Log-Likelihood:                -960.05
No. Observations:                 959   AIC:                             1946.
Df Residuals:                     946   BIC:                             2009.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.0572 

In [ ]:
COLUMNS_FWD = ['Pts','passes','tackles','age','goals','dribbles','touches_att_pen_area','gca']
COLUMNS_FWD += ['games_starts','age','CL','errors']
COLUMNS_FWD += ['isBundesliga']

X = dataFWD[COLUMNS_FWD]
y = np.log(dataFWD['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                     133.5
Date:                Sat, 27 May 2023   Prob (F-statistic):          5.87e-194
Time:                        15:28:55   Log-Likelihood:                -965.02
No. Observations:                 959   AIC:                             1956.
Df Residuals:                     946   BIC:                             2019.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.1533 

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


# MID

In [ ]:
dataMID = data201920_df[data201920_df['position2'].str[:10]=='Midfielder']
# dataMID.head()

In [ ]:
COLUMNS_MID = ['assists', 'carries','miscontrols','passes_completed_short', 'passes_intercepted','pressures_att_3rd', 'through_balls' ,'progressive_passes', 'passes_into_penalty_area']
X = dataMID[COLUMNS_MID]
y = np.log(dataMID['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif)
vif[vif["VIF Factor"] > 10]
print(results.summary())

   VIF Factor                  features
0    4.191594                     const
1    2.161246                   assists
2   13.824149                   carries
3    3.388560               miscontrols
4    8.799036    passes_completed_short
5    3.972132        passes_intercepted
6    3.162704         pressures_att_3rd
7    2.111072             through_balls
8    8.156643        progressive_passes
9    4.517333  passes_into_penalty_area
                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.327
Model:                            OLS   Adj. R-squared:                  0.320
Method:                 Least Squares   F-statistic:                     50.73
Date:                Sat, 27 May 2023   Prob (F-statistic):           6.19e-75
Time:                        15:28:55   Log-Likelihood:                -1189.3
No. Observations:                 950   AIC:                             2399.
Df Resi

In [ ]:
COLUMNS_MID = ['foot_left','assists', 'carries', 'passes_intercepted','pressures_att_3rd', 'through_balls' ,'progressive_passes', 'passes_into_penalty_area']
COLUMNS_MID += ['tackles_won', 'goals', 'Pts','passes_completed_short', 'passes_into_final_third']
X = dataMID[COLUMNS_MID]
y = np.log(dataMID['value'])
result = stepwise_selection(X, y, COLUMNS_MID)
print(result)

刪除特徵：                   1，P-value：0.7185618300
刪除特徵：                   7，P-value：0.7080792226
刪除特徵：                   9，P-value：0.5003918749
刪除特徵：                   0，P-value：0.3977530022
刪除特徵：                   1，P-value：0.2750995298
刪除特徵：                   1，P-value：0.1856517123
刪除特徵：                   6，P-value：0.0722653664
刪除特徵：                   3，P-value：0.1415261824
刪除特徵：                   2，P-value：0.2827651125
['carries', 'through_balls', 'goals', 'Pts']


In [ ]:
result += ['assists','passes']
result += ['games_starts','age','CL','errors']

X = dataMID[result]
X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.586
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     132.8
Date:                Sat, 27 May 2023   Prob (F-statistic):          5.02e-172
Time:                        15:28:56   Log-Likelihood:                -958.80
No. Observations:                 950   AIC:                             1940.
Df Residuals:                     939   BIC:                             1993.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            15.9606      0.172     92.931

# **固定效果**

In [ ]:
COLUMNS_MID = ['carries','pressures_att_3rd','through_balls', 'goals','Pts','assists','passes']
COLUMNS_MID += ['games_starts','age','CL','errors']
COLUMNS_MID += ['isSerieA']
X = dataMID[COLUMNS_MID]
y = np.log(dataMID['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
vif[vif["VIF Factor"] > 10]
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.590
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                     112.4
Date:                Sat, 27 May 2023   Prob (F-statistic):          5.34e-172
Time:                        15:28:56   Log-Likelihood:                -953.87
No. Observations:                 950   AIC:                             1934.
Df Residuals:                     937   BIC:                             1997.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                15.9410      0.17

In [ ]:
COLUMNS_MID = ['carries','pressures_att_3rd','through_balls', 'goals','Pts','assists','passes']
COLUMNS_MID += ['games_starts','age','CL','errors']
COLUMNS_MID += ['isPremierLeague']
X = dataMID[COLUMNS_MID]
y = np.log(dataMID['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
vif[vif["VIF Factor"] > 10]
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.619
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                     127.0
Date:                Sat, 27 May 2023   Prob (F-statistic):          5.67e-187
Time:                        15:28:56   Log-Likelihood:                -918.67
No. Observations:                 950   AIC:                             1863.
Df Residuals:                     937   BIC:                             1926.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                15.9838      0.16

In [ ]:
COLUMNS_MID = ['carries','pressures_att_3rd','through_balls', 'goals','Pts','assists','passes']
COLUMNS_MID += ['games_starts','age','CL','errors']
COLUMNS_MID += ['isLigue1']
X = dataMID[COLUMNS_MID]
y = np.log(dataMID['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
vif[vif["VIF Factor"] > 10]
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.596
Model:                            OLS   Adj. R-squared:                  0.591
Method:                 Least Squares   F-statistic:                     115.3
Date:                Sat, 27 May 2023   Prob (F-statistic):          4.05e-175
Time:                        15:28:57   Log-Likelihood:                -946.53
No. Observations:                 950   AIC:                             1919.
Df Residuals:                     937   BIC:                             1982.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                16.1403      0.17

In [ ]:
COLUMNS_MID = ['carries','pressures_att_3rd','through_balls', 'goals','Pts','assists','passes']
COLUMNS_MID += ['games_starts','age','CL','errors']
COLUMNS_MID += ['isLaLiga']
X = dataMID[COLUMNS_MID]
y = np.log(dataMID['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
vif[vif["VIF Factor"] > 10]
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.587
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     110.8
Date:                Sat, 27 May 2023   Prob (F-statistic):          2.24e-170
Time:                        15:28:57   Log-Likelihood:                -957.69
No. Observations:                 950   AIC:                             1941.
Df Residuals:                     937   BIC:                             2005.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                15.9489      0.17

In [ ]:
COLUMNS_MID = ['carries','pressures_att_3rd','through_balls', 'goals','Pts','assists','passes']
COLUMNS_MID += ['games_starts','age','CL','errors']
COLUMNS_MID += ['isBundesliga']
X = dataMID[COLUMNS_MID]
y = np.log(dataMID['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
vif[vif["VIF Factor"] > 10]
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.586
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     110.4
Date:                Sat, 27 May 2023   Prob (F-statistic):          6.01e-170
Time:                        15:28:57   Log-Likelihood:                -958.70
No. Observations:                 950   AIC:                             1943.
Df Residuals:                     937   BIC:                             2007.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                15.9547      0.17

# DEF

In [ ]:
dataDEF = data201920_df[data201920_df['position2'].str[:8]=='Defender']
# dataDEF.head()

In [ ]:
# COLUMNS_DEF = ['games_starts','age','CL','errors',]
COLUMNS_DEF = ['games_starts','age','CL','errors','clearances','passes_ground','touches_att_pen_area','touches_def_pen_area','aerials_won_pct']
X = dataDEF[COLUMNS_DEF]
# print(X['carries'])
# X.insert(0, 'age', dataDEF['age']**2)
y = np.log(dataDEF['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif)
print(results.summary())

   VIF Factor              features
0   69.611738                 const
1    5.059356          games_starts
2    1.038955                   age
3    1.420608                    CL
4    1.275109                errors
5    4.892414            clearances
6    3.669299         passes_ground
7    1.840507  touches_att_pen_area
8    5.785260  touches_def_pen_area
9    1.258885       aerials_won_pct
                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.532
Method:                 Least Squares   F-statistic:                     199.9
Date:                Sat, 27 May 2023   Prob (F-statistic):          1.02e-252
Time:                        15:28:57   Log-Likelihood:                -1598.4
No. Observations:                1576   AIC:                             3217.
Df Residuals:                    1566   BIC:       

# 加入固定效果

In [ ]:
COLUMNS_DEF = ['games_starts','age','CL','errors','clearances','passes_ground','touches_att_pen_area','touches_def_pen_area','aerials_won_pct']
COLUMNS_DEF += ['isBundesliga']
X = dataDEF[COLUMNS_DEF]
y = np.log(dataDEF['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.532
Method:                 Least Squares   F-statistic:                     180.1
Date:                Sat, 27 May 2023   Prob (F-statistic):          7.58e-252
Time:                        15:28:57   Log-Likelihood:                -1597.7
No. Observations:                1576   AIC:                             3217.
Df Residuals:                    1565   BIC:                             3276.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.7975 

In [ ]:
COLUMNS_DEF = ['games_starts','age','CL','errors','clearances','passes_ground','touches_att_pen_area','touches_def_pen_area','aerials_won_pct']
COLUMNS_DEF += ['isLaLiga']
X = dataDEF[COLUMNS_DEF]
# print(X['carries'])
# X.insert(0, 'age', dataDEF['age']**2)
y = np.log(dataDEF['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.532
Method:                 Least Squares   F-statistic:                     180.1
Date:                Sat, 27 May 2023   Prob (F-statistic):          8.04e-252
Time:                        15:28:57   Log-Likelihood:                -1597.8
No. Observations:                1576   AIC:                             3218.
Df Residuals:                    1565   BIC:                             3277.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.7734 

In [ ]:
COLUMNS_DEF = ['games_starts','age','CL','errors','clearances','passes_ground','touches_att_pen_area','touches_def_pen_area','aerials_won_pct']
COLUMNS_DEF += ['isLigue1']
X = dataDEF[COLUMNS_DEF]
# print(X['carries'])
# X.insert(0, 'age', dataDEF['age']**2)
y = np.log(dataDEF['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.547
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     189.1
Date:                Sat, 27 May 2023   Prob (F-statistic):          7.84e-261
Time:                        15:28:58   Log-Likelihood:                -1576.8
No. Observations:                1576   AIC:                             3176.
Df Residuals:                    1565   BIC:                             3235.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.8344 

In [ ]:
COLUMNS_DEF = ['games_starts','age','CL','errors','clearances','passes_ground','touches_att_pen_area','touches_def_pen_area','aerials_won_pct']
COLUMNS_DEF += ['isPremierLeague']
X = dataDEF[COLUMNS_DEF]
# print(X['carries'])
# X.insert(0, 'age', dataDEF['age']**2)
y = np.log(dataDEF['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.581
Method:                 Least Squares   F-statistic:                     219.8
Date:                Sat, 27 May 2023   Prob (F-statistic):          1.40e-289
Time:                        15:28:58   Log-Likelihood:                -1509.9
No. Observations:                1576   AIC:                             3042.
Df Residuals:                    1565   BIC:                             3101.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.7508 

In [ ]:
COLUMNS_DEF = ['games_starts','age','CL','errors','clearances','passes_ground','touches_att_pen_area','touches_def_pen_area','aerials_won_pct']
COLUMNS_DEF += ['isSerieA']
X = dataDEF[COLUMNS_DEF]
# print(X['carries'])
# X.insert(0, 'age', dataDEF['age']**2)
y = np.log(dataDEF['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     186.9
Date:                Sat, 27 May 2023   Prob (F-statistic):          1.36e-258
Time:                        15:28:58   Log-Likelihood:                -1582.0
No. Observations:                1576   AIC:                             3186.
Df Residuals:                    1565   BIC:                             3245.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   16.8420 

In [ ]:
COLUMNS_DEF = ['games_starts','age','CL','errors','clearances','passes_ground','touches_att_pen_area','touches_def_pen_area','aerials_won_pct']
COLUMNS_DEF += ['isSerieA','isPremierLeague','isLigue1','isLaLiga','isBundesliga']
X = dataDEF[COLUMNS_DEF]
# print(X['carries'])
# X.insert(0, 'age', dataDEF['age']**2)
y = np.log(dataDEF['value'])
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif)
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.591
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     173.5
Date:                Sat, 27 May 2023   Prob (F-statistic):          1.36e-291
Time:                        15:28:58   Log-Likelihood:                -1497.0
No. Observations:                1576   AIC:                             3022.
Df Residuals:                    1562   BIC:                             3097.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   14.0496 

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


# 'isSerieA','isPremierLeague','isLigue1','isLaLiga','isBundesliga'

In [ ]:
COLUMNS_DEF = ['age','ball_recoveries','errors','foot_left','Pts','CL', 'miscontrols', 'clearances', 'pens_conceded', 'xg_net','fouls','aerials_won_pct']
COLUMNS_DEF += ['touches_def_pen_area','touches_att_pen_area', 'tackles_won']
X = dataDEF[COLUMNS_DEF]
y = np.log(dataDEF['value'])
result = stepwise_selection(X, y, COLUMNS_DEF)
print(result)

刪除特徵：                  11，P-value：0.7622174101
刪除特徵：                   7，P-value：0.6535424534
刪除特徵：                   8，P-value：0.2720330309
刪除特徵：                   2，P-value：0.0954190586
刪除特徵：                  10，P-value：0.0818431108
刪除特徵：                   5，P-value：0.1232076662
刪除特徵：                   2，P-value：0.1062322544
['age', 'ball_recoveries', 'Pts', 'CL', 'pens_conceded', 'fouls', 'touches_def_pen_area', 'touches_att_pen_area']


In [ ]:
X = dataDEF[result]
X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif)

print(results.summary())

   VIF Factor              features
0   65.416417                 const
1    1.041450                   age
2    3.394169       ball_recoveries
3    2.338875                   Pts
4    2.291901                    CL
5    1.130878         pens_conceded
6    1.829916                 fouls
7    2.331174  touches_def_pen_area
8    1.568504  touches_att_pen_area
                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.568
Model:                            OLS   Adj. R-squared:                  0.565
Method:                 Least Squares   F-statistic:                     257.1
Date:                Sat, 27 May 2023   Prob (F-statistic):          7.74e-279
Time:                        15:28:59   Log-Likelihood:                -1540.5
No. Observations:                1576   AIC:                             3099.
Df Residuals:                    1567   BIC:                             3147.
Df Model

# GoalKeeper

In [ ]:
dataGK = data201920_df[data201920_df['position2']=='Goalkeeper']
dataGK = dataGK[dataGK['player']!='Emiliano Viviano'] #guy was causing some problems
# dataGK.head()

In [ ]:
# COLUMNS_GK = ['games_starts','age','CL','psnpxg_per_shot_on_target_against','clean_sheets','wins_gk','errors','passes_pct_launched_gk']
COLUMNS_GK = ['games_starts','age','CL','saves','clean_sheets','wins_gk','errors','passes_pct_launched_gk']

# COLUMNS_GK += ['clean_sheetsm', 'passes_pct_launched_gk', 'psnpxg_per_shot_on_target_against', 'draws_gk','CL', 'age']
X = dataGK[COLUMNS_GK]
y = np.log(dataGK['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif[1:])
print(results.summary())

NameError: ignored

# 加入隊伍固定效果

In [ ]:
COLUMNS_GK = ['games_starts','age','CL','psnpxg_per_shot_on_target_against','clean_sheets','wins_gk','errors','passes_pct_launched_gk','isPremierLeague']
# COLUMNS_GK += ['clean_sheetsm', 'passes_pct_launched_gk', 'psnpxg_per_shot_on_target_against', 'draws_gk','CL', 'age']
X = dataGK[COLUMNS_GK]
y = np.log(dataGK['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif[1:])
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.687
Model:                            OLS   Adj. R-squared:                  0.676
Method:                 Least Squares   F-statistic:                     61.90
Date:                Sat, 27 May 2023   Prob (F-statistic):           5.24e-59
Time:                        15:28:59   Log-Likelihood:                -216.61
No. Observations:                 264   AIC:                             453.2
Df Residuals:                     254   BIC:                             489.0
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [ ]:
COLUMNS_GK = ['games_starts','age','CL','psnpxg_per_shot_on_target_against','clean_sheets','wins_gk','errors','passes_pct_launched_gk','isLigue1']
# COLUMNS_GK += ['clean_sheetsm', 'passes_pct_launched_gk', 'psnpxg_per_shot_on_target_against', 'draws_gk','CL', 'age']
X = dataGK[COLUMNS_GK]
y = np.log(dataGK['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif[1:])
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.647
Method:                 Least Squares   F-statistic:                     54.59
Date:                Sat, 27 May 2023   Prob (F-statistic):           2.10e-54
Time:                        15:28:59   Log-Likelihood:                -227.77
No. Observations:                 264   AIC:                             475.5
Df Residuals:                     254   BIC:                             511.3
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [ ]:
COLUMNS_GK = ['games_starts','age','CL','psnpxg_per_shot_on_target_against','clean_sheets','wins_gk','errors','passes_pct_launched_gk','isLaLiga']
# COLUMNS_GK += ['clean_sheetsm', 'passes_pct_launched_gk', 'psnpxg_per_shot_on_target_against', 'draws_gk','CL', 'age']
X = dataGK[COLUMNS_GK]
y = np.log(dataGK['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif[1:])
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.624
Model:                            OLS   Adj. R-squared:                  0.611
Method:                 Least Squares   F-statistic:                     46.89
Date:                Sat, 27 May 2023   Prob (F-statistic):           4.23e-49
Time:                        15:28:59   Log-Likelihood:                -240.66
No. Observations:                 264   AIC:                             501.3
Df Residuals:                     254   BIC:                             537.1
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [ ]:
COLUMNS_GK = ['games_starts','age','CL','psnpxg_per_shot_on_target_against','clean_sheets','wins_gk','errors','passes_pct_launched_gk','isBundesliga']
# COLUMNS_GK += ['clean_sheetsm', 'passes_pct_launched_gk', 'psnpxg_per_shot_on_target_against', 'draws_gk','CL', 'age']
X = dataGK[COLUMNS_GK]
y = np.log(dataGK['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif[1:])
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.616
Method:                 Least Squares   F-statistic:                     47.89
Date:                Sat, 27 May 2023   Prob (F-statistic):           8.13e-50
Time:                        15:28:59   Log-Likelihood:                -238.92
No. Observations:                 264   AIC:                             497.8
Df Residuals:                     254   BIC:                             533.6
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [ ]:
COLUMNS_GK = ['games_starts','age','CL','psnpxg_per_shot_on_target_against','clean_sheets','wins_gk','errors','passes_pct_launched_gk','isSerieA']
X = dataGK[COLUMNS_GK]
y = np.log(dataGK['value'])

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
# print(vif[1:])
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.612
Method:                 Least Squares   F-statistic:                     47.01
Date:                Sat, 27 May 2023   Prob (F-statistic):           3.48e-49
Time:                        15:28:59   Log-Likelihood:                -240.45
No. Observations:                 264   AIC:                             500.9
Df Residuals:                     254   BIC:                             536.7
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [ ]:
COLUMNS_GK = ['clean_sheets', 'saves','own_goals_against_gk', 'GA','errors','games_starts', 'pens_conceded' , 'wins_gk', 'passes_other_body', 'dribble_tackles_pct']
COLUMNS_GK += ['passes_pct_launched_gk', 'psnpxg_per_shot_on_target_against','CL', 'age']
X = dataGK[COLUMNS_GK]
y = np.log(dataGK['value'])
result = stepwise_selection(X, y, COLUMNS_GK)
print(result)

刪除特徵：                   6，P-value：0.8284326716
刪除特徵：                   1，P-value：0.8283943591
刪除特徵：                   1，P-value：0.8103840063
刪除特徵：                   1，P-value：0.7657989956
刪除特徵：                   5，P-value：0.7357570560
刪除特徵：                   4，P-value：0.4732247985
刪除特徵：                   1，P-value：0.3439907362
刪除特徵：                   3，P-value：0.3262312431
刪除特徵：                   1，P-value：0.1180505668
['clean_sheets', 'wins_gk', 'psnpxg_per_shot_on_target_against', 'CL', 'age']


In [ ]:
# clean_sheetsm = 守門員無失球次數除以上場時間(分鐘)（由於分母固定，係數看起來很大）
# psnpxg_per_shot_on_target_against = 守門員每次被對方射正門的進球數
# passes_pct_launched_gk = 傳出超過40碼傳球的比例
features = ['errors']
features += result

X = dataGK[features]

X = sm.add_constant(X)
model = sm.OLS(y, X)

results = model.fit()
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
print(vif[1:])
print(results.summary())

   VIF Factor                           features
1    1.101917                             errors
2    3.496138                       clean_sheets
3    4.178419                            wins_gk
4    1.023527  psnpxg_per_shot_on_target_against
5    1.473855                                 CL
6    1.016462                                age
                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                     69.03
Date:                Sat, 27 May 2023   Prob (F-statistic):           8.69e-51
Time:                        15:29:00   Log-Likelihood:                -243.16
No. Observations:                 264   AIC:                             500.3
Df Residuals:                     257   BIC:                             525.3
Df Model:                